In [ ]:
!pip install transformers datasets torch accelerate sdv

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sdv.single_table import CTGANSynthesizer  # For synthetic data generation
import pandas as pd
import json


In [ ]:
import json

# Load Spider dataset (Update path if needed)
with open("/content/spider_extracted/spider_data/tables.json", "r") as f:
    tables_data = json.load(f)

with open("/content/spider_extracted/spider_data/train_spider.json", "r") as f:
    train_data = json.load(f)

# Check schema structure
print(json.dumps(tables_data[:1], indent=2))
print(json.dumps(train_data[:1], indent=2))


[
  {
    "column_names": [
      [
        -1,
        "*"
      ],
      [
        0,
        "perpetrator id"
      ],
      [
        0,
        "people id"
      ],
      [
        0,
        "date"
      ],
      [
        0,
        "year"
      ],
      [
        0,
        "location"
      ],
      [
        0,
        "country"
      ],
      [
        0,
        "killed"
      ],
      [
        0,
        "injured"
      ],
      [
        1,
        "people id"
      ],
      [
        1,
        "name"
      ],
      [
        1,
        "height"
      ],
      [
        1,
        "weight"
      ],
      [
        1,
        "home town"
      ]
    ],
    "column_names_original": [
      [
        -1,
        "*"
      ],
      [
        0,
        "Perpetrator_ID"
      ],
      [
        0,
        "People_ID"
      ],
      [
        0,
        "Date"
      ],
      [
        0,
        "Year"
      ],
      [
        0,
        "Location"
      ],
      [
        0,


In [ ]:
def convert_to_orass(schema):
    tables = schema["table_names"]
    columns = schema["column_names"][1:]  # Skip wildcard
    relationships = schema.get("foreign_keys", [])

    formatted_schema = "Schema:\n"
    for table in tables:
        formatted_schema += f"  Object: {table}\n"
        table_columns = [col[1] for col in columns if col[0] == tables.index(table)]
        formatted_schema += f"    Attributes: {', '.join(table_columns)}\n"

    for fk in relationships:
        parent_table, parent_col = columns[fk[0]]
        child_table, child_col = columns[fk[1]]
        formatted_schema += f"  Relationship: {child_table} → {parent_table} (FK: {child_col})\n"

    return formatted_schema

# Convert a sample schema
print(convert_to_orass(tables_data[0]))

Schema:
  Object: perpetrator
    Attributes: perpetrator id, people id, date, year, location, country, killed, injured
  Object: people
    Attributes: people id, name, height, weight, home town
  Relationship: 1 → 0 (FK: name)



In [ ]:
def convert_to_orass(schema):
    tables = schema["table_names"]
    columns = schema["column_names"][1:]  # Skip wildcard
    relationships = schema.get("foreign_keys", [])

    formatted_schema = "Schema:\n"
    for table in tables:
        formatted_schema += f"  Object: {table}\n"
        table_columns = [col[1] for col in columns if col[0] == tables.index(table)]
        formatted_schema += f"    Attributes: {', '.join(table_columns)}\n"

    for fk in relationships:
        parent_table, parent_col = columns[fk[0]]
        child_table, child_col = columns[fk[1]]
        formatted_schema += f"  Relationship: {child_table} → {parent_table} (FK: {child_col})\n"

    return formatted_schema

# Convert and print a sample schema in ORA-SS format
sample_orass_schema = convert_to_orass(tables_data[0])
print("ORA-SS Schema Format:\n", sample_orass_schema)

ORA-SS Schema Format:
 Schema:
  Object: perpetrator
    Attributes: perpetrator id, people id, date, year, location, country, killed, injured
  Object: people
    Attributes: people id, name, height, weight, home town
  Relationship: 1 → 0 (FK: name)



In [ ]:
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
import pandas as pd
import random

# Extract a sample table from Spider
table_name = tables_data[0]["table_names"][0]  # First table
columns = [col[1] for col in tables_data[0]["column_names"] if col[0] == 0]  # Columns of the first table

# Create a sample dataset (Fake sample data)
real_data = pd.DataFrame({
    col: [random.randint(1, 100) for _ in range(10)] for col in columns  # Generate random values
})

# Define metadata for SDV
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(real_data)

# Initialize SDV synthesizer with metadata
synthesizer = CTGANSynthesizer(metadata)

# Train SDV model
synthesizer.fit(real_data)

# Generate synthetic data
synthetic_data = synthesizer.sample(10)
print("Synthetic Table Data:\n", synthetic_data)


/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Synthetic Table Data:
    perpetrator id  people id      date  year  location  country  killed  \
0              39         68   3005065    67        58       25      25   
1              34         59    449512    30        89       74      28   
2              37         66  16507995    41        10       23      33   
3              36         76  16083568    80        33      438      47   
4              34         51   3914937    80        76       96      42   
5              71         23  10593927    66        33      909      83   
6              70         78  14311281    40        65      202      80   
7              56         38   5309139    90        28        9      72   
8              40         81  15944639    27        95      674      43   
9              36         26   8345955    51        77       34      76   

   injured  
0        3  
1       25  
2       16  
3       52  
4        9  
5        9  
6        6  
7        5  
8        4  
9       38  


In [ ]:
import random

def generate_text_sql_pairs(synthetic_data, schema):
    text_sql_pairs = []

    for _, row in synthetic_data.iterrows():
        table_name = schema["table_names"][0]
        col_name = random.choice([col[1] for col in schema["column_names"] if col[0] == 0])

        question = f"What is the value of {col_name} in {table_name}?"
        sql_query = f"SELECT {col_name} FROM {table_name} LIMIT 5"

        text_sql_pairs.append({"question": question, "sql_query": sql_query})

    return text_sql_pairs

# Generate Text-to-SQL pairs
synthetic_text_sql = generate_text_sql_pairs(synthetic_data, tables_data[0])
print("Synthetic Text-to-SQL Pairs:\n", synthetic_text_sql[:5])


Synthetic Text-to-SQL Pairs:
 [{'question': 'What is the value of country in perpetrator?', 'sql_query': 'SELECT country FROM perpetrator LIMIT 5'}, {'question': 'What is the value of injured in perpetrator?', 'sql_query': 'SELECT injured FROM perpetrator LIMIT 5'}, {'question': 'What is the value of country in perpetrator?', 'sql_query': 'SELECT country FROM perpetrator LIMIT 5'}, {'question': 'What is the value of country in perpetrator?', 'sql_query': 'SELECT country FROM perpetrator LIMIT 5'}, {'question': 'What is the value of location in perpetrator?', 'sql_query': 'SELECT location FROM perpetrator LIMIT 5'}]


In [ ]:
import random

def generate_advanced_sql(synthetic_data, schema):
    """Generate complex SQL queries with WHERE, JOIN, and GROUP BY."""
    text_sql_pairs = []

    table_name = schema["table_names"][0]  # Use the first table for now
    columns = [col[1] for col in schema["column_names"] if col[0] == 0]  # Columns from first table

    for _, row in synthetic_data.iterrows():
        col_name = random.choice(columns)  # Pick a random column
        value = row[col_name]  # Get a random value from synthetic data

        # Choose a random SQL pattern
        query_type = random.choice(["WHERE", "JOIN", "GROUP BY", "COUNT", "SUM"])

        if query_type == "WHERE":
            question = f"What are the records where {col_name} is {value}?"
            sql_query = f"SELECT * FROM {table_name} WHERE {col_name} = '{value}';"

        elif query_type == "JOIN" and len(schema["table_names"]) > 1:
            # Create a JOIN query if multiple tables exist
            second_table = schema["table_names"][1]
            question = f"Join {table_name} with {second_table} and return all records."
            sql_query = f"SELECT * FROM {table_name} INNER JOIN {second_table} ON {table_name}.id = {second_table}.id;"

        elif query_type == "GROUP BY":
            question = f"Group records by {col_name} and count occurrences."
            sql_query = f"SELECT {col_name}, COUNT(*) FROM {table_name} GROUP BY {col_name};"

        elif query_type == "COUNT":
            question = f"How many records exist in {table_name}?"
            sql_query = f"SELECT COUNT(*) FROM {table_name};"

        elif query_type == "SUM":
            question = f"What is the total sum of {col_name}?"
            sql_query = f"SELECT SUM({col_name}) FROM {table_name};"

        text_sql_pairs.append({"question": question, "sql_query": sql_query})

    return text_sql_pairs

# Generate synthetic SQL queries
advanced_text_sql = generate_advanced_sql(synthetic_data, tables_data[0])

# Display samples
print("Sample Advanced Text-to-SQL Pairs:\n", advanced_text_sql[:5])


Sample Advanced Text-to-SQL Pairs:
 [{'question': 'What are the records where country is 25?', 'sql_query': "SELECT * FROM perpetrator WHERE country = '25';"}, {'question': 'Group records by perpetrator id and count occurrences.', 'sql_query': 'SELECT perpetrator id, COUNT(*) FROM perpetrator GROUP BY perpetrator id;'}, {'question': 'Join perpetrator with people and return all records.', 'sql_query': 'SELECT * FROM perpetrator INNER JOIN people ON perpetrator.id = people.id;'}, {'question': 'Join perpetrator with people and return all records.', 'sql_query': 'SELECT * FROM perpetrator INNER JOIN people ON perpetrator.id = people.id;'}, {'question': 'Group records by year and count occurrences.', 'sql_query': 'SELECT year, COUNT(*) FROM perpetrator GROUP BY year;'}]


In [ ]:
# Load Spider train data
with open("/content/spider_extracted/spider_data/train_spider.json", "r") as f:
    train_spider_data = json.load(f)

# Merge synthetic queries
merged_train_data = train_spider_data + advanced_text_sql

# Save merged dataset
with open("merged_train_spider.json", "w") as f:
    json.dump(merged_train_data, f, indent=4)

print("Merged dataset saved as 'merged_train_spider.json'")

Merged dataset saved as 'merged_train_spider.json'


In [ ]:
from transformers import T5Tokenizer
import json

# Load merged dataset
with open("/content/merged_train_spider.json", "r") as f:
    train_data = json.load(f)

# Initialize tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-large")

# Convert to T5 format
def preprocess_function(example):
    """
    Preprocesses a single example from the dataset.

    Handles cases where 'sql_query' might be missing. If missing,
    it uses 'query' instead, which is likely the key used in
    the original Spider dataset.
    """
    inputs = f"Schema: {example['question']}"
    # Use 'query' if 'sql_query' is not found
    targets = example.get("sql_query", example.get("query", ""))
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
train_dataset = [preprocess_function(example) for example in train_data] # Change map to a list comprehension to process individual examples

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
import torch
torch.cuda.empty_cache()

# Kill all previous processes using GPU
!nvidia-smi
!kill -9 $(nvidia-smi | awk '$5=="python"{print $3}')

Sat Mar 15 10:51:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P0             30W /   70W |     162MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

# Load model
model = T5ForConditionalGeneration.from_pretrained("t5-large").to("cuda")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_t5_sql",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=3,
    save_steps=1000,
    evaluation_strategy="no",
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
    optim="adafactor",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: syedaumaizaunsa (syedaumaizaunsa-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,inf


TrainOutput(global_step=657, training_loss=nan, metrics={'train_runtime': 9105.4458, 'train_samples_per_second': 2.31, 'train_steps_per_second': 0.072, 'total_flos': 4.5388051513344e+16, 'train_loss': nan, 'epoch': 2.9905848787446505})

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the fine-tuned model
model = T5ForConditionalGeneration.from_pretrained("/content/fine_tuned_t5_sql/checkpoint-657").to("cuda")
tokenizer = T5Tokenizer.from_pretrained("t5-large")

# Load validation dataset
with open("/content/validation_spider.json", "r") as f:
    val_data = json.load(f)

# Function to compute accuracy
def evaluate_model(model, dataset, tokenizer):
    exact_match, logical_form_match, execution_match = 0, 0, 0
    total = len(dataset)

    for sample in dataset:
        question = f"Schema: {sample['query']}"
        true_sql = sample["query"]

        # Tokenize input
        inputs = tokenizer(question, return_tensors="pt", truncation=True).to("cuda")

        # Generate SQL query
        output_ids = model.generate(**inputs)
        pred_sql = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Exact Match (EM)
        if pred_sql.strip().lower() == true_sql.strip().lower():
            exact_match += 1

        # Logical Form Accuracy (LF) - Ignores formatting differences
        if set(pred_sql.lower().split()) == set(true_sql.lower().split()):
            logical_form_match += 1

        # Execution Accuracy (EX) - Requires actual database execution (Optional)
        # execution_match += execute_and_compare(pred_sql, true_sql)

    print(f"✅ Exact Match Accuracy (EM): {exact_match / total:.2%}")
    print(f"✅ Logical Form Accuracy (LF): {logical_form_match / total:.2%}")
    # print(f"✅ Execution Accuracy (EX): {execution_match / total:.2%}") # Uncomment if database is available

# Run Evaluation
evaluate_model(model, val_data, tokenizer)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyError: 'query'

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the fine-tuned model
model = T5ForConditionalGeneration.from_pretrained("/content/fine_tuned_t5_sql/checkpoint-657").to("cuda")
tokenizer = T5Tokenizer.from_pretrained("t5-large")

# Load validation dataset
with open("/content/validation_spider.json", "r") as f:
    val_data = json.load(f)

# Function to compute accuracy
def evaluate_model(model, dataset, tokenizer):
    exact_match, logical_form_match, execution_match = 0, 0, 0
    total = len(dataset)

    for sample in dataset:
        question = f"Schema: {sample['query']}"
        # Use 'query' if 'sql_query' is not found
        true_sql = sample.get("sql_query", sample.get("query", ""))

        inputs = tokenizer(question, return_tensors="pt", truncation=True).to("cuda")

        output_ids = model.generate(**inputs)
        pred_sql = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        if pred_sql.strip().lower() == true_sql.strip().lower():
            exact_match += 1

        if set(pred_sql.lower().split()) == set(true_sql.lower().split()):
            logical_form_match += 1

    print(f"✅ Exact Match Accuracy (EM): {exact_match / total:.2%}")
    print(f"✅ Logical Form Accuracy (LF): {logical_form_match / total:.2%}")

evaluate_model(model, val_data, tokenizer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Exact Match Accuracy (EM): 0.00%
✅ Logical Form Accuracy (LF): 0.00%


In [42]:
def evaluate_model_debug(model, dataset, tokenizer):
    model.eval()  # Set model to evaluation mode
    total = len(dataset)
    exact_match = 0

    for sample in dataset:
        question = f"Schema: {sample['query']}"
        true_sql = sample["query"]

        # Tokenize input
        inputs = tokenizer(question, return_tensors="pt", truncation=True).to("cuda")

        # Generate SQL query
        output_ids = model.generate(**inputs)
        pred_sql = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Print sample comparisons
        print("Question:", sample["question"])
        print("Expected SQL:", true_sql)
        print("Generated SQL:", pred_sql)

        # Check exact match
        if pred_sql.strip().lower() == true_sql.strip().lower():
            exact_match += 1

    print(f"\n✅ Exact Match Accuracy: {exact_match / total:.2%}")

# Run debug evaluation
evaluate_model_debug(model, val_data, tokenizer)


Question: Find the personal name, family name, and author ID of the course author that teaches the most courses.
Expected SQL: SELECT T1.personal_name ,  T1.family_name ,  T2.author_id FROM Course_Authors_and_Tutors AS T1 JOIN Courses AS T2 ON T1.author_id  =  T2.author_id GROUP BY T2.author_id ORDER BY COUNT(*) DESC LIMIT 1
Generated SQL: T2.author_id DESC LIMIT 1 SELECT T1.author_i
Question: Show the country names and the corresponding number of players.
Expected SQL: SELECT Country_name ,  COUNT(*) FROM country AS T1 JOIN match_season AS T2 ON T1.Country_id  =  T2.Country GROUP BY T1.Country_name
Generated SQL: FROM country AS T1 JOIN match_season AS T2 ON T2.Country
Question: What are the name, origin and owner of each program?
Expected SQL: SELECT name ,  origin ,  OWNER FROM program
Generated SQL: ,,,, ,, , ,
Question: What are the all games score and location of the school called Clemson?
Expected SQL: SELECT t2.All_Games ,  t1.location FROM university AS t1 JOIN basketball_matc

KeyboardInterrupt: 